# Training
on entraine le réseau de neurones pour les hyperparamètres

In [1]:
# importation
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import cv2 as cv
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
# local
from MyResNet.myfunc import Physics
from MyResNet.myfunc import MyMatmul
from MyResNet.model import MyModel

### Paramètres physiques

In [2]:
a = 1
p = 1
nx = 10
tensor_list = Physics(a,p,nx)
Tt          = MyMatmul(tensor_list[1].T)
mass        = 1
U           = torch.FloatTensor(np.ones(nx))

In [9]:
def CreateLoader(folder):
    """
    According to the mode, creates the appropriate loader 
    for the training and validation sets.
    """
    dfx     = pd.read_csv(folder+'/'+'data_lisse.csv', sep=',',header=None)
    dfy     = pd.read_csv(folder+'/'+'data_blurred.csv', sep=',',header=None)
    #
    x_tensor= torch.tensor(dfx.values).view(-1,1,200)
    y_tensor= torch.tensor(dfy.values).view(-1,1,200)
    #
    dataset = TensorDataset(x_tensor, y_tensor)
    l = len(dataset)
    m = 2*l//3
    train_dataset, val_dataset = random_split(dataset, [m, l-m])
    #
    train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)
    val_loader   = DataLoader(val_dataset, batch_size=1, shuffle=False)
    return train_loader, val_loader

In [10]:
folder = './MyResNet/Dataset'
t, v = CreateLoader(folder)

In [ ]:
def train(self,model,nb_epochs):
    """
    Trains iRestNet.
    """      
    # to store results
    loss_epochs  =  np.zeros(nb_epochs)
    loss_train   =  np.zeros(nb_epochs)
    loss_val     =  np.zeros(nb_epochs)
    loss_min_val =  float('Inf')
    self.CreateLoader()
    # defines the optimizer
    lr_i        = 0.01
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad,self.parameters()),lr=lr_i)

    #==========================================================================================================
    # trains for several epochs
    for epoch in range(0,self.nb_epochs[0]): 
        # sets training mode
        self.model.train()
        # modifies learning rate
        if epoch>0:
            lr_i      = lr_i*0.9 
            optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad,self.parameters()), lr=lr_i)
        # goes through all minibatches
        for i,minibatch in enumerate(self.train_loader,0):
            [x_true, x_blurred] = minibatch    # get the minibatch
            x_true    = Variable(x_true.type(self.dtype),requires_grad=False)
            x_init    = Variable(x_blurred.type(self.dtype),requires_grad=False)
            # ATTENTION : on ne calcule pas le gradient en fonction de la deuxième sortie (à réfléchir)
            Ttx_init  = Tt(x_init).detach()     # do not compute gradient
            x_pred    = self.model(x_init,Ttx_init,self.mode) 
                    
            # Computes and prints loss
            loss                = self.loss_fun(x_pred, x_true)
            loss_epochs[epoch] += torch.Tensor.item(loss)
            sys.stdout.write('\r(%d, %3d) minibatch loss: %5.4f '%(epoch,i,torch.Tensor.item(loss)))
                    
            # sets the gradients to zero, performs a backward pass, and updates the weights.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
                
             # for statistics
            loss_train[:,:,epoch] += compute_stat(x_true, x_blurred, x_pred, self.size_train)

        #==========================================================================================================
        # training is finished
        print('-----------------------------------------------------------------')
        print('Training is done.')
        self.SaveLoss(epoch,loss_epochs,self.mode)
        print('-----------------------------------------------------------------')

    